In [1]:
import numpy as np
import chainer
import chainer.links as L
import chainer.functions as F

/home/mayu-ot/miniconda3/envs/py36chainer/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import strip_punctuation

データ

In [3]:
dialogue = [
"Hi guys.",
"Hey!",
"Did you get my mail.",
"Yeah, right here. How was Nebraska?",
"Oh, better than North Dakota! (Pause) I guess that joke’s only funny in Nebraska.",
]

前処理（ここはいろいろ頑張る方法があるところ）

In [4]:
doc = [strip_punctuation(d).lower().split() for d in dialogue]
doc

[['hi', 'guys'],
 ['hey'],
 ['did', 'you', 'get', 'my', 'mail'],
 ['yeah', 'right', 'here', 'how', 'was', 'nebraska'],
 ['oh',
  'better',
  'than',
  'north',
  'dakota',
  'pause',
  'i',
  'guess',
  'that',
  'joke’s',
  'only',
  'funny',
  'in',
  'nebraska']]

In [5]:
dictionary = Dictionary(doc)

In [6]:
batch = [list(map(dictionary.token2id.get, d)) for d in doc]
batch = [np.asarray(x).astype('i') for x in batch]
batch

[array([0, 1], dtype=int32),
 array([2], dtype=int32),
 array([3, 4, 5, 6, 7], dtype=int32),
 array([ 8,  9, 10, 11, 12, 13], dtype=int32),
 array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 13],
       dtype=int32)]

In [7]:
emb = L.EmbedID(len(dictionary), 100)
lstm_0 = L.NStepLSTM(1, 100, 100, 0.0)
lstm_1 = L.NStepLSTM(1, 100, 100, 0.0)

In [8]:
word_embs = [emb(x) for x in batch]
[w.shape for w in word_embs]

[(2, 100), (1, 100), (5, 100), (6, 100), (14, 100)]

In [9]:
h_0, c_0, y_0 = lstm_0(None, None, word_embs)
h_0 = h_0[0]

In [10]:
h_1, c_1, y_1 = lstm_1(None, None, [h_0])
y_1 = y_1[0]

In [11]:
y_1.shape

(5, 100)